### Imports

In [50]:
from prophet import Prophet

import pandas as pd
import os
import plotly.graph_objects as go
import pandas as pd


import numpy as np
from sklearn.metrics import mean_squared_error



### Read data

In [51]:
folder_path = './data/'
files = os.listdir(folder_path)
print(files)

list_to_concat = []
for file in files[:2]:
    if 'txt' in file:
        list_to_concat.append(pd.read_csv(f'./data/{file}', sep=','))

['AFKS_daily_010114_010124.csv', 'AFKS_daily_010114_010124.txt']


In [52]:
def column_name_preprocesing(df):
    df = df.copy()
    df.rename(columns={'<OPEN>': 'open', '<HIGH>': 'high', '<LOW>': 'low', '<CLOSE>': 'close', '<VOL>': 'volume', '<TICKER>':'symbol','<DATE>':'datetime'}, inplace=True)
    columns_to_keep = ['symbol', 'datetime', 'open', 'close', 'high', 'low', 'volume']
    df = df[columns_to_keep]
    # df.sort_values(by = ['symbol','datetime'], inplace = True)
    # df['change_in_price'] = df['close'].diff()
    return df

In [53]:
result = pd.concat(list_to_concat, ignore_index=True)
result = column_name_preprocesing(result)

In [54]:
result.drop(columns=['symbol', 'open', 'high','low','volume'], inplace = True)

In [55]:
result.rename(columns={'datetime': 'ds',
                   'close': 'y'},
          inplace=True)

In [56]:
result['ds'] = pd.to_datetime(result['ds'], format='%Y%m%d')


In [57]:
predictions = 200

train_df = result[:-predictions] 
test_df = result[-predictions:] 

### Hyperparameters

In [35]:
interval_widths = [round(i * 0.01, 2) for i in range(1, 100, 10)]
changepoint_prior_scales = [round(i * 0.01, 2) for i in range(1, 100, 10)]
seasonality_prior_scales = [round(i * 0.01, 2) for i in range(1, 100, 10)]

In [37]:
best_mse = np.inf
best_interval_width = 0
best_changepoint_prior_scale = 0
best_seasonality_prior_scale = 0

for interval_width in interval_widths:
    for changepoint_prior_scale in changepoint_prior_scales:
        for seasonality_prior_scale in seasonality_prior_scales:
            model = Prophet(interval_width=interval_width,changepoint_prior_scale=changepoint_prior_scale,seasonality_prior_scale=seasonality_prior_scale, weekly_seasonality=True)
            model.fit(train_df)
            forecast = model.predict(test_df.drop(columns=['y']))[["ds","yhat"]]
            mse = mean_squared_error(test_df['y'], forecast['yhat'])

            if mse<best_mse:
                best_mse = mse
                best_interval_width = interval_width
                best_changepoint_prior_scale = changepoint_prior_scale
                best_seasonality_prior_scale = seasonality_prior_scale



18:29:34 - cmdstanpy - INFO - Chain [1] start processing
18:29:34 - cmdstanpy - INFO - Chain [1] done processing
18:29:35 - cmdstanpy - INFO - Chain [1] start processing
18:29:35 - cmdstanpy - INFO - Chain [1] done processing
18:29:35 - cmdstanpy - INFO - Chain [1] start processing
18:29:35 - cmdstanpy - INFO - Chain [1] done processing
18:29:36 - cmdstanpy - INFO - Chain [1] start processing
18:29:36 - cmdstanpy - INFO - Chain [1] done processing
18:29:36 - cmdstanpy - INFO - Chain [1] start processing
18:29:36 - cmdstanpy - INFO - Chain [1] done processing
18:29:37 - cmdstanpy - INFO - Chain [1] start processing
18:29:37 - cmdstanpy - INFO - Chain [1] done processing
18:29:37 - cmdstanpy - INFO - Chain [1] start processing
18:29:37 - cmdstanpy - INFO - Chain [1] done processing
18:29:38 - cmdstanpy - INFO - Chain [1] start processing
18:29:38 - cmdstanpy - INFO - Chain [1] done processing
18:29:38 - cmdstanpy - INFO - Chain [1] start processing
18:29:38 - cmdstanpy - INFO - Chain [1]

### Results

In [58]:
a, b, c = best_interval_width, best_changepoint_prior_scale,best_seasonality_prior_scale

model = Prophet(interval_width=a,changepoint_prior_scale=b,seasonality_prior_scale=c, weekly_seasonality=True)
model.fit(train_df)
forecast_all = model.predict(result[:].drop(columns=['y']))[["ds","yhat"]]
forecast = model.predict(test_df.drop(columns=['y']))[["ds","yhat"]]

19:17:12 - cmdstanpy - INFO - Chain [1] start processing
19:17:12 - cmdstanpy - INFO - Chain [1] done processing


In [61]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=result['ds'],
                        y=result['y']
                        , name="true"))
fig.add_trace(go.Scatter(x=forecast_all['ds'],
                        y=forecast_all['yhat']
                        , name="predicted"))
fig.add_trace(go.Scatter(x=forecast['ds'],
                        y=forecast['yhat']
                        , name="predicted"))

fig.update_layout(title_text="Prophet",
                  height=400, width=1000).show()
